In [1]:
# Notebook: 04_Telemetry_Ingestion.ipynb

import fastf1
import s3fs
import pandas as pd
import numpy as np

# --- 1. Configure S3 (MinIO) Connection ---
endpoint_url = 'http://minio:9000'
s3 = s3fs.S3FileSystem(
    client_kwargs={'endpoint_url': endpoint_url},
    key='minioadmin',
    secret='minioadmin'
)

# --- 2. Load Session Data ---
# Enable FastF1 cache
fastf1.Cache.enable_cache('/home/jovyan/.cache/fastf1') 

print("Loading session data for Qualifying...")
# Using 2024 Bahrain Qualifying
session = fastf1.get_session(2024, 'Bahrain', 'Q')
session.load(telemetry=True, laps=True, weather=False)

# --- 3. Get Fastest Laps for Two Drivers ---
print("Getting fastest laps for VER and LEC...")
ver_lap = session.laps.pick_driver('VER').pick_fastest()
lec_lap = session.laps.pick_driver('LEC').pick_fastest()

# --- 4. Get Telemetry Data for These Laps ---
print("Fetching telemetry data...")
ver_tel = ver_lap.get_telemetry().add_distance()
lec_tel = lec_lap.get_telemetry().add_distance()

# Add a 'Driver' column so we can group them later
ver_tel['Driver'] = 'VER'
lec_tel['Driver'] = 'LEC'

# --- 5. Combine and Clean Data ---
print("Combining and cleaning data...")
# Combine the two dataframes
telemetry_df = pd.concat([ver_tel, lec_tel])

# === THE PARQUET FIX (Apply what we learned) ===
# 1. Convert timedelta columns
timedelta_cols = telemetry_df.select_dtypes(include=['timedelta64[ns]']).columns
if not timedelta_cols.empty:
    print(f"Converting {len(timedelta_cols)} timedelta columns to seconds...")
    for col in timedelta_cols:
        telemetry_df.loc[:, col] = telemetry_df[col].dt.total_seconds()

# 2. Convert datetime columns
datetime_cols = telemetry_df.select_dtypes(include=['datetime64[ns]']).columns
if not datetime_cols.empty:
    print(f"Converting {len(datetime_cols)} datetime columns to string...")
    for col in datetime_cols:
        telemetry_df.loc[:, col] = telemetry_df[col].astype(str)
        
# 3. Fill NaNs (important for Spark)
# Telemetry has NaNs for Brake, RPM etc. Let's fill them.
numeric_cols = telemetry_df.select_dtypes(include=[np.number]).columns
telemetry_df[numeric_cols] = telemetry_df[numeric_cols].fillna(0)
# === END FIX ===

print(f"Total telemetry rows: {len(telemetry_df)}")
print(telemetry_df.head())

# --- 6. Save to MinIO Data Lake ---
file_path = 's3://raw-data/2024_bahrain_Q_telemetry.parquet'
print(f"Saving telemetry data to {file_path}...")

with s3.open(file_path, 'wb') as f:
    telemetry_df.to_parquet(
        f,
        index=False,
        version='2.4',
        coerce_timestamps='us'
    )

print("Telemetry ingestion complete!")

Loading session data for Qualifying...


core           INFO 	Loading data for Bahrain Grand Prix - Qualifying [v3.6.1]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '16', '63', '55', '11', '14', '4', '81', '44', '27', '22', '18', '23', '3', '20', '77', '24', '2', '31', '10']
/opt/conda/lib/python3.11/site-packages/fastf1/core.py:3183: FutureWarning: pick_driver is deprecated and will be removed in a future release. Use pick_drivers inste

Getting fastest laps for VER and LEC...
Fetching telemetry data...
Combining and cleaning data...
Converting 2 timedelta columns to seconds...
Converting 1 datetime columns to string...
Total telemetry rows: 1389
                     Date  SessionTime DriverAhead  DistanceToDriverAhead  \
2 2024-03-01 16:59:09.798     4354.125                          50.658889   
3 2024-03-01 16:59:09.837     4354.164                          50.658889   
4 2024-03-01 16:59:09.889     4354.216                          50.658889   
5 2024-03-01 16:59:10.016     4354.343                          50.658889   
6 2024-03-01 16:59:10.049     4354.376          81              50.658889   

    Time           RPM       Speed  nGear  Throttle  Brake  DRS  \
2  0.000  10854.482953  297.241668      7     100.0  False   12   
3  0.039  10777.133333  297.566667      7     100.0  False   12   
4  0.091  10674.000000  298.000000      8     100.0  False   12   
5  0.218  10739.881283  300.381251      8     100.0  Fal